<a href="https://colab.research.google.com/github/serciex/lane-change/blob/main/v6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing Prerequisites

In [1]:
! pip install pygame
! pip install highway-env
! pip install stable-baselines3
! pip install tdqm
! pip install gym
! pip install "numpy>=1.19.0,<2.0.0"
! pip install random
! pip install torch
! pip install matplotlib
! pip install highway_env
! pip install tensorboard
! pip install imghdr
! pip install stable-baselines3[extra]
! pip install tdqm
! pip install ipywidgets
! pip install rich
! pip install wandb
! pip install wandb[media]


ERROR: Could not find a version that satisfies the requirement random (from versions: none)
ERROR: No matching distribution found for random
ERROR: Could not find a version that satisfies the requirement imghdr (from versions: none)
ERROR: No matching distribution found for imghdr
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 75.2 MB/s eta 0:00:00


Testing

In [2]:
#Import Libraries
import math
import gymnasium
import highway_env
from matplotlib import pyplot as plt
import pygame
import numpy as np
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import tqdm
import gym
from random import randint
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from collections import deque
import random
import torch.optim as optim
import tqdm
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from IPython.display import display


# Workaround for gym compatibility
if not hasattr(np, 'bool8'):
    np.bool8 = np.bool_

%matplotlib inline

/tmp/ipython-input-2-782785167.py:28: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not hasattr(np, 'bool8'):


Environment Wrapper

In [3]:
import math
import numpy as np
import gymnasium

class ENVwrapper(gymnasium.Wrapper):
    def __init__(self, env, parameters):
        super().__init__(env)

        # Define observation and action spaces
        self.observation_space = gymnasium.spaces.Box(
            low=np.array([
                -1000, -50, 0, -5, -np.pi, -1, 3.0, -np.pi
            ]),
            high=np.array([
                10000, 50, 50, 5, np.pi, 1, 4.0, np.pi
            ]),
            dtype=np.float32)

        # 1D action space for steering only (highway-env expects array)
        self.action_space = gymnasium.spaces.Box(
            low=-0.5, high=0.5, shape=(1,), dtype=np.float32
        )

        # Initialize parameters (only reward weights needed now)
        _, _, self.reward_weights, self.following_gap = parameters
        self.w1, self.w2, self.w3 = self.reward_weights
        self.following_gap_threshold = self.following_gap

        # State tracking
        self.current_obs = None
        self.previous_obs = None
        self.last_lane = None
        self.gap_controller_checked = False  # Keep for backward compatibility if needed
        self.target_id = 0

        # Cooldown approach attributes
        self.cooldown_counter = 0
        self.target_lane_id = None  # Will be set in first call

        # Yaw rate storage
        self.current_yaw_rate = 0.0
        self.previous_yaw_rate = 0.0

    def lane_check(self):
        """
        Determines which adjacent lane offers the largest front gap for a lane change.
        Uses cooldown approach - only evaluates lane change after 10 calls when in target lane.
        """
        obs = self.current_obs
        ego = self.unwrapped.vehicle
        current_lane_id = self.unwrapped.vehicle.lane_index[2]

        # Initialize cooldown counter if not exists
        if not hasattr(self, 'cooldown_counter'):
            self.cooldown_counter = 0

        # Initialize target lane if not exists (start with current lane)
        if not hasattr(self, 'target_lane_id') or self.target_lane_id is None:
            self.target_lane_id = current_lane_id

        # Reset cooldown counter when lane changes
        if self.last_lane is not None and current_lane_id != self.last_lane:
            self.cooldown_counter = 0

        # Check if vehicle is in target lane
        vehicle_in_target_lane = (current_lane_id == self.target_lane_id)

        # Only proceed with lane change evaluation if:
        # 1. Vehicle is in target lane AND
        # 2. Cooldown counter has reached threshold (10 calls)
        if vehicle_in_target_lane and self.cooldown_counter >= 10:

            # Get front gap from highway-env observation
            gap_current_front = obs[1][1] if len(obs) > 1 and obs[1][0] > 0 else float('inf')

            # Get right lane front gap
            current_lane = list(ego.lane_index)
            lane_right = (current_lane[0], current_lane[1], current_lane[2] + 1) if current_lane[2] < 2 else ego.lane_index
            neighbours_right = self.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_right)
            gap_right_front = neighbours_right[0].position[0] - ego.position[0] if neighbours_right and neighbours_right[0] else -float('inf')

            # Get left lane front gap
            lane_left = (current_lane[0], current_lane[1], current_lane[2] - 1) if current_lane[2] > 0 else ego.lane_index
            neighbours_left = self.unwrapped.road.neighbour_vehicles(ego, lane_index=lane_left)
            gap_left_front = neighbours_left[0].position[0] - ego.position[0] if neighbours_left and neighbours_left[0] else -float('inf')

            # Find best lane
            front_gaps = [gap_current_front, gap_right_front, gap_left_front]
            best_index = np.argmax(front_gaps)

            if best_index == 1:  # Right lane is best
                direction = 1
                gap_right_follower = ego.position[0] - neighbours_right[1].position[0] if len(neighbours_right) > 1 and neighbours_right[1] else float('inf')
                if gap_right_follower >= self.following_gap_threshold:
                    self.target_id = direction
                    self.target_lane_id = current_lane_id + 1  # Update target lane
                    self.cooldown_counter = 0  # Reset counter after lane change decision
                else:
                    self.target_id = 0  # Stay in current lane

            elif best_index == 2:  # Left lane is best
                direction = -1
                gap_left_follower = ego.position[0] - neighbours_left[1].position[0] if len(neighbours_left) > 1 and neighbours_left[1] else float('inf')
                if gap_left_follower >= self.following_gap_threshold:
                    self.target_id = direction
                    self.target_lane_id = current_lane_id - 1  # Update target lane
                    self.cooldown_counter = 0  # Reset counter after lane change decision
                else:
                    self.target_id = 0  # Stay in current lane

            else:  # Current lane is best
                self.target_id = 0  # Stay in current lane
                # Don't reset counter - continue counting in current lane

        else:
            # Vehicle not in target lane OR cooldown not reached
            # Return command to move towards target lane or stay
            if current_lane_id < self.target_lane_id:
                self.target_id = 1  # Move right towards target
            elif current_lane_id > self.target_lane_id:
                self.target_id = -1  # Move left towards target
            else:
                self.target_id = 0  # In target lane, but cooldown not reached

        # Increment cooldown counter only when in target lane
        if vehicle_in_target_lane:
            self.cooldown_counter += 1

        self.last_lane = current_lane_id
        return self.target_id

    def calculate_yaw_rate_from_heading(self, obs_current, obs_previous):
        """
        Calculate yaw rate from heading difference
        """
        if obs_previous is None:
            return 0.0

        # Raw heading from highway-env observation
        heading_current = obs_current[0][5]
        heading_previous = obs_previous[0][5]

        # Calculate heading difference with wraparound
        heading_diff = heading_current - heading_previous
        heading_diff = np.arctan2(np.sin(heading_diff), np.cos(heading_diff))

        # Convert to yaw rate: heading difference × policy frequency
        policy_frequency = self.unwrapped.config['policy_frequency']
        yaw_rate = heading_diff * policy_frequency

        return yaw_rate

    def calculate_yaw_acceleration(self):
        """
        Calculate yaw acceleration from yaw rate difference
        """
        policy_frequency = self.unwrapped.config['policy_frequency']
        yaw_acceleration = (self.current_yaw_rate - self.previous_yaw_rate) * policy_frequency

        return yaw_acceleration

    def reward_function(self, obs_old, obs_new):
        """
        Reward function using heading-based yaw rate and acceleration
        """
        if obs_old is None:
            return 0.0

        target_id = self.lane_check()
        w1, w2, w3 = self.reward_weights

        # Convert direction command to actual target lane
        current_lane_id = self.unwrapped.vehicle.lane_index[2]

        if target_id == +1:  # Move right
            actual_target_lane = min(2, current_lane_id + 1)
        elif target_id == -1:  # Move left
            actual_target_lane = max(0, current_lane_id - 1)
        else:  # target_id == 0, stay in current lane
            actual_target_lane = current_lane_id

        # Use current vehicle lane structure and set actual target lane
        current_lane_index = list(self.unwrapped.vehicle.lane_index)
        current_lane_index[2] = actual_target_lane
        self.target_lane = tuple(current_lane_index)

        target_lane_object = self.unwrapped.road.network.get_lane(self.target_lane)
        vehicle_s, _ = self.unwrapped.vehicle.lane.local_coordinates(self.unwrapped.vehicle.position)
        _, self.delta_lat_deviation = target_lane_object.local_coordinates(self.unwrapped.vehicle.position)

        # Calculate yaw rate and acceleration from heading differences
        new_yaw_rate = self.calculate_yaw_rate_from_heading(obs_new, obs_old)
        yaw_acceleration = self.calculate_yaw_acceleration()

        # Update stored yaw rate values for next iteration
        self.previous_yaw_rate = self.current_yaw_rate
        self.current_yaw_rate = new_yaw_rate

        # Reward calculations - all penalties
        acce_reward = -w1 * abs(yaw_acceleration)           # Penalty for yaw acceleration
        rate_reward = -w2 * abs(self.current_yaw_rate)      # Penalty for yaw rate

        # Time reward: zero when in target lane, penalty for lateral deviation when not in target lane
        current_lane_id = self.unwrapped.vehicle.lane_index[2]
        vehicle_in_target_lane = (current_lane_id == self.target_lane_id)

        if vehicle_in_target_lane:
            time_reward = 0.0  # Zero penalty when in target lane
        else:
            time_reward = -w3 * abs(self.delta_lat_deviation)  # Penalty for lateral deviation when not in target lane

        # Total reward
        self.reward = acce_reward + rate_reward + time_reward

        return self.reward

    def step(self, agent_action):
        # Ensure action is 1D array for highway-env
        if isinstance(agent_action, np.ndarray) and agent_action.shape == ():
            agent_action = np.array([agent_action.item()], dtype=np.float32)
        elif not isinstance(agent_action, np.ndarray):
            agent_action = np.array([float(agent_action)], dtype=np.float32)

        # Clip action to valid range
        agent_action = np.clip(agent_action, -0.5, 0.5)

        obs_old = self.current_obs
        vehicle = self.unwrapped.vehicle

        # Prior velocity for acceleration calculation
        v_prev = obs_old[0][3] if obs_old is not None else 0

        # Step environment with steering action (1D array)
        obs, _, done, truncated, info = super().step(agent_action)

        # Save highway-env observation for reward function
        self.previous_obs = obs_old
        self.current_obs = obs

        custom_reward = self.reward_function(obs_old, obs)

        # Improved termination conditions
        crashed = info.get('crashed', False) or vehicle.crashed
        off_road = not vehicle.on_road
        done = done or truncated or crashed or off_road

        # Extract ego vehicle data for agent observation
        ego_data = obs[0]  # [presence, x, y, vx, vy, heading]

        x = ego_data[1]      # x position (longitudinal)
        y = ego_data[2]      # y position (lateral)
        vx = ego_data[3]     # vx (longitudinal velocity)
        theta = ego_data[5]  # heading
        lane_width = vehicle.lane.width

        lane_id = self.lane_check()

        # Safe curvature calculation
        try:
            local_s = vehicle.lane.local_coordinates(vehicle.position)[0]
            local_s = np.clip(local_s, 0, vehicle.lane.length - 1)
            self_curvature = vehicle.lane.heading_at(local_s)
        except:
            self_curvature = 0.0

        acceleration = (vx - v_prev) * self.unwrapped.config['policy_frequency']

        # Create custom agent observation
        agent_observation = np.array([
            x, y, vx, acceleration, theta, lane_id, lane_width, self_curvature
        ], dtype=np.float32)

        return agent_observation, custom_reward, done, truncated, info

    def reset(self, **kwargs):
        obs, info = super().reset(**kwargs)

        # Save highway-env observation for reward function
        self.current_obs = obs
        self.previous_obs = None

        # Reset state tracking
        vehicle = self.unwrapped.vehicle
        self.last_lane = None
        self.gap_controller_checked = False

        # Reset cooldown approach attributes
        self.cooldown_counter = 0
        self.target_lane_id = self.unwrapped.vehicle.lane_index[2]  # Start with current lane

        # Reset yaw rate storage
        self.current_yaw_rate = 0.0
        self.previous_yaw_rate = 0.0

        # Extract ego vehicle data for agent observation
        ego_data = obs[0]  # [presence, x, y, vx, vy, heading]

        x = ego_data[1]      # x position (longitudinal)
        y = ego_data[2]      # y position (lateral)
        vx = ego_data[3]     # vx (longitudinal velocity)
        theta = ego_data[5]  # heading

        lane_width = self.unwrapped.vehicle.lane.width
        lane_id = self.lane_check()

        # Safe curvature calculation
        try:
            local_s = vehicle.lane.local_coordinates(vehicle.position)[0]
            local_s = np.clip(local_s, 0, vehicle.lane.length - 1)
            self_curvature = vehicle.lane.heading_at(local_s)
        except:
            self_curvature = 0.0

        acceleration = 0  # Initial acceleration is zero

        # Create custom agent observation
        agent_observation = np.array([
            x, y, vx, acceleration, theta, lane_id, lane_width, self_curvature
        ], dtype=np.float32)

        return agent_observation, info

Training Loop

In [ ]:
import gymnasium as gym
import highway_env
from stable_baselines3 import PPO
import numpy as np
import torch
import wandb
from stable_baselines3.common.callbacks import BaseCallback

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

def make_env():
    env = gym.make("highway-v0", render_mode="rgb_array")
    env.unwrapped.configure({
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 15,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "absolute": False,
        },
        "action": {
            "type": "ContinuousAction",
            "longitudinal": False,
            "lateral": True
        },
        "lanes_count": 3,
        "vehicles_count": 50,
        "duration": 40,
    })

    reward_weights = [1.0, 1.0, 0.5]
    following_gap = 30.0
    parameters = (None, None, reward_weights, following_gap)

    env = ENVwrapper(env, parameters)
    return env

class ProgressCallback(BaseCallback):
    def __init__(self, check_freq=1000, video_freq=500):
        super().__init__()
        self.check_freq = check_freq
        self.video_freq = video_freq
        self.episode_rewards = []
        self.current_episode_reward = 0

    def _on_step(self):
        if 'rewards' in self.locals:
            self.current_episode_reward += self.locals['rewards']

        if 'dones' in self.locals and self.locals['dones']:
            self.episode_rewards.append(self.current_episode_reward)

            wandb.log({
                "episode_reward": self.current_episode_reward,
                "episode": len(self.episode_rewards),
                "timestep": self.n_calls,
                "mean_reward_last_100": np.mean(self.episode_rewards[-100:]) if len(self.episode_rewards) >= 100 else np.mean(self.episode_rewards)
            })

            self.current_episode_reward = 0

        if self.n_calls % self.check_freq == 0:
            print(f"Step: {self.n_calls}, Episodes: {len(self.episode_rewards)}")

        if self.n_calls % self.video_freq == 0 and self.n_calls > 0:
            self._log_agent_video()

        return True

    def _log_agent_video(self):
        try:
            print("Recording agent video...")
            video_env = make_env()

            frames = []
            obs, _ = video_env.reset()
            episode_reward = 0

            for step in range(200):
                action, _ = self.model.predict(obs, deterministic=True)
                obs, reward, done, truncated, info = video_env.step(action)
                episode_reward += reward

                frame = video_env.render()
                if frame is not None and len(frame.shape) == 3:
                    frames.append(frame)

                if done or truncated:
                    break

            video_env.close()

            if frames and len(frames) > 5:
                frames_array = np.array(frames)
                if len(frames_array.shape) == 4:
                    frames_array = frames_array.transpose(0, 3, 1, 2)

                wandb.log({
                    "videos/agent_performance": wandb.Video(
                        frames_array,
                        fps=10,
                        format="mp4"
                    ),
                    "videos/episode_reward": episode_reward,
                    "timestep": self.n_calls
                })
                print(f"Video logged: {len(frames)} frames, reward: {episode_reward:.2f}")

        except Exception as e:
            print(f"Video logging failed: {e}")

# Hyperparameters
learning_rate = 3e-4
batch_size = 64
n_steps = 2048
gamma = 0.99
clip_range = 0.2
gae_lambda = 0.95
policy_net_arch = [256, 128]
value_net_arch = [128, 64]
total_timesteps = 100000

wandb.init(
    project="highway-ppo-training",
    name="highway-ppo-agent-only",
    config={
        "total_timesteps": total_timesteps,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "n_steps": n_steps,
        "gamma": gamma,
        "clip_range": clip_range,
        "gae_lambda": gae_lambda,
        "policy_net_arch": policy_net_arch,
        "value_net_arch": value_net_arch,
        "reward_weights": [1.0, 1.0, 0.5],
        "following_gap": 30.0
    },
    tags=["highway-env", "ppo", "steering-only"]
)

env = make_env()

model = PPO(
    "MlpPolicy",
    env,
    learning_rate=learning_rate,
    batch_size=batch_size,
    n_steps=n_steps,
    gamma=gamma,
    clip_range=clip_range,
    gae_lambda=gae_lambda,
    policy_kwargs={
        'net_arch': dict(pi=policy_net_arch, vf=value_net_arch),
        'activation_fn': torch.nn.Tanh
    },
    verbose=1,
    tensorboard_log="./ppo_logs/",
    device=device
)

print("Training...")
callback = ProgressCallback(check_freq=500, video_freq=1000)

model.learn(
    total_timesteps=total_timesteps,
    tb_log_name="highway_steering_training",
    progress_bar=True,
    callback=callback
)

model.save("ppo_highway_steering")
print("Training done!")

print("Testing...")
test_env = make_env()
test_rewards = []
obs, _ = test_env.reset()

for i in range(100):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, truncated, info = test_env.step(action)

    if done or truncated:
        test_rewards.append(reward)
        obs, _ = test_env.reset()

test_env.close()

if test_rewards:
    wandb.log({
        "mean_test_reward": np.mean(test_rewards),
        "max_test_reward": np.max(test_rewards)
    })

wandb.finish()
print("Done!")

Using device: cuda


/usr/local/lib/python3.11/dist-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


...

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

/usr/local/lib/python3.11/dist-packages/ipywidgets/widgets/widget_output.py:111: DeprecationWarning: 
Kernel._parent_header is deprecated in ipykernel 6. Use .get_parent()
  if ip and hasattr(ip, 'kernel') and hasattr(ip.kernel, '_parent_header'):

In [ ]:
# Load the trained model (assumes model variable already exists from training)
# If model doesn't exist, uncomment the line below:
# model = PPO.load("ppo_highway_steering")

def make_env():
    env = gym.make("highway-v0", render_mode="rgb_array")
    env.unwrapped.configure({
        "observation": {
            "type": "Kinematics",
            "vehicles_count": 15,
            "features": ["presence", "x", "y", "vx", "vy", "heading"],
            "absolute": False,
        },
        "action": {
            "type": "ContinuousAction",
            "longitudinal": False,
            "lateral": True
        },
        "lanes_count": 3,
        "vehicles_count": 50,
        "duration": 40,
    })

    reward_weights = [1.0, 1.0, 0.5]
    following_gap = 30.0
    parameters = (None, None, reward_weights, following_gap)

    env = ENVwrapper(env, parameters)
    return env

# This function is removed - video is now created during testing

# Test the model and create video of the testing
def test_model_with_video(num_episodes=5):
    test_env = make_env()
    test_rewards = []
    all_frames = []

    print(f"Testing model for {num_episodes} episodes...")

    # Initialize wandb for video logging
    wandb.init(
        project="highway-ppo-testing",
        name="highway-ppo-test-video"
    )

    for episode in range(num_episodes):
        obs, _ = test_env.reset()
        episode_reward = 0
        steps = 0
        episode_frames = []

        for step in range(200):
            action, _ = model.predict(obs, deterministic=True)
            obs, reward, done, truncated, info = test_env.step(action)
            episode_reward += reward
            steps += 1

            # Record frame
            frame = test_env.render()
            if frame is not None and len(frame.shape) == 3:
                episode_frames.append(frame)

            if done or truncated:
                break

        test_rewards.append(episode_reward)
        all_frames.extend(episode_frames)
        print(f"Episode {episode + 1}: Reward = {episode_reward:.2f}, Steps = {steps}")

    test_env.close()

    # Log video of testing
    if all_frames and len(all_frames) > 5:
        frames_array = np.array(all_frames)
        if len(frames_array.shape) == 4:
            frames_array = frames_array.transpose(0, 3, 1, 2)

        wandb.log({
            "videos/testing_episodes": wandb.Video(
                frames_array,
                fps=10,
                format="mp4"
            ),
            "mean_test_reward": np.mean(test_rewards),
            "max_test_reward": np.max(test_rewards)
        })

        print(f"Video logged: {len(all_frames)} frames from {num_episodes} episodes")

    print(f"\nTest Results:")
    print(f"Mean reward: {np.mean(test_rewards):.2f}")
    print(f"Max reward: {np.max(test_rewards):.2f}")
    print(f"Min reward: {np.min(test_rewards):.2f}")
    print(f"Std reward: {np.std(test_rewards):.2f}")

    wandb.finish()
    return test_rewards

# Run tests with video recording
test_rewards = test_model_with_video(num_episodes=3)